Imports y configuraciones de ambiente

In [ ]:
import torch
import numpy as np
import random
import numpy as np
import gymnasium

Checkeo de devices

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Running on {DEVICE}")
print("Cuda Available:", torch.cuda.is_available())

Setting de seeds

In [ ]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

Validacion del ambiente

In [ ]:
ENV_NAME = "CartPole-v1"

env = gymnasium.make(ENV_NAME, render_mode="rgb_array")

print("Actions #",env.action_space)
print(env.observation_space.shape)
env.reset()
next_state, reward, terminated, truncated, info = env.step(action=0)

print(f"{next_state.shape},\n {reward},\n {terminated},\n {info}")

In [ ]:
from replay_memory import ReplayMemory

In [ ]:
from dqn_cnn_model import DQN_CNN_Model

env = gymnasium.make(ENV_NAME)

test_net = DQN_CNN_Model(env.observation_space.shape[0], env.action_space.n).to(DEVICE)

In [ ]:
def process_state(obs, device):
    return torch.tensor(obs, device=device).unsqueeze(0)

#Hiperparámetros de entrenamiento del agente DQN
TOTAL_STEPS = 1000000
EPISODES = 10000
STEPS = 200

EPSILON_INI = 1
EPSILON_MIN = 0.1
EPSILON_DECAY = 0.998
EPISODE_BLOCK = 500
EPSILON_TIME = 200


BATCH_SIZE = 64
BUFFER_SIZE = 10000

GAMMA = 0.999
LEARNING_RATE = 1e-3

In [ ]:
from dqn_agent import DQNAgent
env = gymnasium.make(ENV_NAME)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
net = DQN_CNN_Model(env.observation_space.shape[0], env.action_space.n).to(DEVICE)

In [ ]:
#gym_env, model, obs_processing_func, memory_buffer_size, batch_size, learning_rate, gamma, epsilon_i, epsilon_f, epsilon_anneal_time
agent = DQNAgent(env, net, process_state, BUFFER_SIZE, BATCH_SIZE, 
                LEARNING_RATE, GAMMA, epsilon_i= EPSILON_INI, 
                epsilon_f=EPSILON_MIN, epsilon_anneal_time=EPSILON_TIME,
                epsilon_decay = EPSILON_DECAY, episode_block = EPISODE_BLOCK, device=DEVICE)

In [ ]:
rewards = agent.train(EPISODES, STEPS, TOTAL_STEPS, writer_name = ENV_NAME)

In [ ]:
import matplotlib.pyplot as plt

average_range = EPISODE_BLOCK
episode_ticks = int(len(rewards) / average_range)

avg_rewards = np.array(rewards).reshape((episode_ticks, average_range))
avg_rewards = np.mean(avg_rewards, axis=1)

plt.plot(range(len(avg_rewards)), avg_rewards)